In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [4]:
data_path = "C:/Users/kzhan/Desktop/skin_cancer"
train_path = os.path.join(data_path + "/train")
validation_path = os.path.join(data_path + "/test")

batch_size, epochs = 32, 15
image_size = (128, 128)
class_mode = 'binary'

training_dir = tf.keras.preprocessing.image_dataset_from_directory(
    directory = train_path, subset = "training", seed = 123, image_size = image_size,
    color_mode = 'rgb', validation_split = 0.2, batch_size = batch_size)

validation_dir = tf.keras.preprocessing.image_dataset_from_directory(
    directory = validation_path, subset = "validation", seed = 123, image_size = image_size,
    color_mode = 'rgb', validation_split = 0.2, batch_size = batch_size)

Found 2637 files belonging to 2 classes.
Using 2110 files for training.
Found 660 files belonging to 2 classes.
Using 132 files for validation.


In [7]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

weights_file = "inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

pretrained_model = tf.keras.applications.inception_v3.InceptionV3(include_top = False, 
weights = None, input_shape = (128, 128, 3))

pretrained_model.load_weights(weights_file)

for layer in pretrained_model.layers:
    layer.trainable = False

pretrained_model.summary()

161[0][0]             
__________________________________________________________________________________________________
conv2d_163 (Conv2D)             (None, 6, 6, 192)    147456      average_pooling2d_15[0][0]       
__________________________________________________________________________________________________
batch_normalization_154 (BatchN (None, 6, 6, 192)    576         conv2d_154[0][0]                 
__________________________________________________________________________________________________
batch_normalization_157 (BatchN (None, 6, 6, 192)    576         conv2d_157[0][0]                 
__________________________________________________________________________________________________
batch_normalization_162 (BatchN (None, 6, 6, 192)    576         conv2d_162[0][0]                 
__________________________________________________________________________________________________
batch_normalization_163 (BatchN (None, 6, 6, 192)    576         conv2d_163[0][0]     

In [8]:
last_layer = pretrained_model.get_layer('mixed7')
last_layer_output = last_layer.output

In [15]:
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import Model

x = tf.keras.layers.Flatten()(last_layer_output)
x = tf.keras.layers.Dense(1024, activation = tf.nn.relu)(x)
x = tf.keras.layers.Dense(0.2)(x)
x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.Model(pretrained_model.inputs, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.001), 
loss = tf.keras.losses.BinaryCrossentropy(from_logits = True), metrics = ['accuracy'])

In [16]:
model.summary()

              
__________________________________________________________________________________________________
activation_139 (Activation)     (None, 6, 6, 160)    0           batch_normalization_139[0][0]    
__________________________________________________________________________________________________
conv2d_135 (Conv2D)             (None, 6, 6, 160)    122880      mixed4[0][0]                     
__________________________________________________________________________________________________
conv2d_140 (Conv2D)             (None, 6, 6, 160)    179200      activation_139[0][0]             
__________________________________________________________________________________________________
batch_normalization_135 (BatchN (None, 6, 6, 160)    480         conv2d_135[0][0]                 
__________________________________________________________________________________________________
batch_normalization_140 (BatchN (None, 6, 6, 160)    480         conv2d_140[0][0]             

In [12]:
history = model.fit(training_dir, validation_data = validation_dir, epochs = epochs)

Epoch 1/15
19/66 [=======>......................] - ETA: 27s - loss: 0.6931 - accuracy: 0.5447

KeyboardInterrupt: 

In [ ]:
s